In [6]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import TweetTokenizer

# Cargar los datos en un DataFrame de pandas
movies_df = pd.read_csv('./data/MovieReviews.csv', sep=',')

# Función para tokenizar los tweets
nltk.download('punkt')
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

# Descargando las stopwords
nltk.download('stopwords')
stop_words = list(stopwords.words('spanish')) + list(stopwords.words('english'))

# Eliminar la columna "Unnamed: 0"
def eliminar_columna(X):
    return X.drop('Unnamed: 0', axis=1)

# Definir las funciones de preprocesamiento
def preprocesamiento(text):
    # Eliminar todos los caracteres no alfanuméricos y convertir a minúsculas
    clean_text = re.sub(r'\W+', ' ', text).lower()
    return clean_text

# Convertir la variable objetivo a valores numéricos
def variable_objetivo(X):
    return 0 if X == 'negativo' else 1


# Crear el pipeline
text_pipeline = Pipeline([
    ('preprocessing', ColumnTransformer(transformers=[
        ('eliminar_columna', FunctionTransformer(eliminar_columna), ['Unnamed: 0']),
        ('preprocesamiento', FunctionTransformer(lambda x: x.apply(preprocesamiento)), 'review_es'),
        ('variable_objetivo', FunctionTransformer(lambda x: x.apply(variable_objetivo)), ['sentimiento']),
    ], remainder='passthrough')),
    ('vectorizer', CountVectorizer(tokenizer=tokenizer, stop_words=stop_words, lowercase=True)),
    ('classifier', RandomForestClassifier(random_state=2))
])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(movies_df['review_es'], movies_df['sentimiento'], test_size=0.2, random_state=42)

# Ajustar el pipeline a los datos de entrenamiento
text_pipeline.fit(X_train, y_train)

# Evaluar el pipeline en los datos de prueba
score = text_pipeline.score(X_test, y_test)
print("Accuracy:", score)


FileNotFoundError: [Errno 2] No such file or directory: '/data/MovieReviews.csv'